# hash code - Practice Problem

## Script Parameters

In [ ]:
file_name = 'a_example.in'
slice_types = [(1,1), (1,2),(1,3),(1,4),(1,5),(1,6),(2,1),(2,2),(2,3),(3,1),(3,2),(4,1),(5,1),(6,1)]
file_name = 'b_small.in'
slice_types = [(1,1), (1,2),(1,3),(1,4),(1,5),(2,1),(2,2),(3,1),(4,1),(5,1)]
#file_name = 'c_medium.in'
#slice_types = [(1,1), (1,8),(1,9),(1,10),(1,11),(1,12),(2,4),(2,5),(2,6),(3,3),(3,4),(4,2),(4,3),(5,2),(6,2),(8,1),(9,1),(10,1),(11,1),(12,1)]
#file_name = 'd_big.in'
#slice_types = [(1,1), (1,12),(1,13),(1,14),(2,6),(2,7),(3,4),(4,3),(6,2),(7,2),(12,1),(13,1),(14,1)]

## Input data

In [ ]:
pizza = []
with open(file_name) as file:
    first_line = file.readline()
    first_line = first_line.split(' ')
    first_line = [int(x) for x in first_line]
    for line in file:
        line = line.replace('\n','')
        line = list(line)
        line = [1 if x == 'T' else 0 for x in line]
        pizza.append(line)
r, c, l, h = first_line[0], first_line[1], first_line[2], first_line[3]

In [ ]:
#pizza

In [ ]:
assert len(pizza) == r

In [ ]:
for row in pizza:
    assert len(row) == c

## Help functions

In [ ]:
def pizza_slice(matrix, position, size):
    if position[0] + size[0] > len(matrix):
        return None
    if position[1] + size[1] > len(matrix[0]):
        return None
    try:
        result = [row[position[1]:position[1]+size[1]] for row in matrix[position[0]:position[0]+size[0]]]
    except IndexError:
        return None
    return result

In [ ]:
pizza_slice(pizza, (0,0), (3,5))

In [ ]:
pizza_slice(pizza, (1,1), (2,4))

In [ ]:
pizza_slice([[1, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]], (1,4), (2,1))

In [ ]:
assert pizza_slice([[1, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]], (1,4), (2,3)) is None

In [ ]:
def has_ingredients(matrix):
    if matrix is None:
        return False
    zero_count = 0
    one_count = 0
    for row in matrix:
        zero_count += row.count(0)
        one_count += row.count(1)
        if zero_count >= l and one_count >= l:
            return True
    return False

In [ ]:
#assert has_ingredients(pizza_slice([[1, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]], (0,0), (1,5))) == False
#assert has_ingredients(pizza_slice([[1, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 1, 1, 1, 1]], (0,0), (2,3))) == True

In [ ]:
def set_slice_type(matrix, position, slice_type, type_index):
    for i in range(position[0], position[0] + slice_type[0]):
        for j in range(position[1], position[1] + slice_type[1]):
            matrix[i][j] = type_index
    return matrix

In [ ]:
#print(set_slice_type([[0] * c for _ in range(r)], (2,1), (1,3), 4))

In [ ]:
def next_position(matrix):
    for i, row in enumerate(matrix):
        for j, _ in enumerate(row):
            if matrix[i][j] == 0:
                return (i,j)
    return None

In [ ]:
assert next_position([[6, 6, 0, 0, 0], [6, 6, 0, 0, 0], [0, 0, 0, 0, 0]]) == (0,2)
assert next_position([[6, 6, 7, 7, 7], [6, 6, 0, 0, 0], [0, 0, 0, 0, 0]]) == (1,2)
assert next_position([[6, 6, 7, 7, 7], [6, 6, 5, 5, 5], [0, 0, 0, 0, 0]]) == (2,0)
assert next_position([[6, 6, 7, 7, 7], [6, 6, 5, 5, 5], [3, 3, 3, 2, 2]]) == None

In [ ]:
def not_sliced(matrix):
    result = 0
    for row in matrix:
        for element in row:
            if element == 0:
                result += 1
    return result

In [ ]:
assert not_sliced([[6, 6, 0, 0, 0], [6, 6, 0, 0, 0], [0, 0, 0, 0, 0]]) == 11
assert not_sliced([[6, 6, 7, 7, 7], [6, 6, 5, 5, 5], [3, 3, 3, 2, 2]]) == 0

In [ ]:
def no_option(matrix):
    result = 0
    for row in matrix:
        for element in row:
            if element == -1:
                result += 1
    return result

## Slice tree

In [ ]:
import copy
import datetime

class SliceTree(object):
    def __init__(self, slice_type, type_index, position, parent):
        self.type = slice_type
        self.index = type_index
        self.position = position
        self.parent = parent
        self.children = []
        if parent is not None:
            parent_coverage = copy.deepcopy(self.parent.coverage)
            self.coverage = set_slice_type(parent_coverage, self.position, self.type, self.index)
        else:
            self.coverage = [[0] * c for _ in range(r)]
        coverage_not_sliced = not_sliced(self.coverage)
        coverage_no_option = no_option(self.coverage)
        self.score = 1 - (coverage_not_sliced + coverage_no_option) / (r*c)
        if coverage_not_sliced == r*c:
            self.partial_score = 1
        else:
            self.partial_score = 1 - coverage_no_option / ((r*c) - coverage_not_sliced)

    def add_childs(self):
        childs_position = next_position(self.coverage)
        if childs_position is not None:
            for i, slice_type in enumerate(slice_types):
                if has_ingredients(pizza_slice(pizza, childs_position, slice_type)):
                    child = SliceTree(slice_type, i, childs_position, self)
                    self.children.append(child)
                    child.add_childs()
            if len(self.children) == 0:
                no_children = SliceTree((1,1), -1, childs_position, self)
                self.children.append(no_children)
        else:
            output_name = file_name + datetime.datetime.now().strftime('_%Y%m%d_%H%M%S_%f') + f'_score_{self.score:.2f}.txt'
            with open(output_name, 'w') as file:
                for row in self.coverage:
                    file.write(str(row) + '\n')

In [ ]:
tree = SliceTree(None, 0, (0,0), None)
tree.add_childs()
#for child_1 in tree.children:
#    child_1.add_childs()
#    for child_2 in child_1.children:
#        child_2.add_childs()
#        for child_3 in child_2.children:
#            child_3.add_childs()

In [ ]:
#for child in tree.children:
#    print(f'{child.type} -> {child.coverage}\t\t\t\t\t\t{child.score:7.2%} {child.partial_score:7.2%}')
#    for child_1 in child.children:
#        print(f'\t{child_1.type} -> {child_1.coverage}\t\t\t\t\t{child_1.score:7.2%} {child_1.partial_score:7.2%}')
#        for child_2 in child_1.children:
#            print(f'\t\t{child_2.type} -> {child_2.coverage}\t\t\t\t{child_2.score:7.2%} {child_2.partial_score:7.2%}')
#            for child_3 in child_2.children:
#                print(f'\t\t\t{child_3.type} -> {child_3.coverage}\t\t\t{child_3.score:7.2%} {child_3.partial_score:7.2%}')